# Data Pre-Processing

In [ ]:
!pip install fastparquet

In [2]:
import os
import sys
import pandas as pd

In [29]:
my_root = "/home/jbobro/final_project/W266_Final_Project"

In [ ]:
# This array will hold all raw abstracts (w/ no association to year etc)
abstracts = []
file_key = []
rootdir = f"{my_root}/Data/ABSTRACT_DATA/"
latest_file = ''

# Loop through directories to grab all of the files
for subdir, dirs, files in os.walk(rootdir):
    for filename in files:
        f = os.path.join(subdir, filename)
        # Open the file (w/ latin1 encoding to forgive speical characters)
        with open(f, 'r', encoding='latin1') as file:
            # grab file key that we will use to join with Bag of Words
            file_key.append(f[-12:][:-4])
            # Create abstract variable to hold abstract after we parse the file
            abstract = []
            all_text = file.readlines()
            # Cut out white space
            output = [line.strip() for line in all_text]
            # Isolate abstract by parsing the file and grabbing all text only after the "Abstract" section
            is_abstract = False
            for val in output:
                if is_abstract:
                    abstract.append(val)
                if val.startswith('Abstract'):
                      is_abstract = True
            abstracts.append(''.join(abstract))

In [21]:
file_ids_auth = []
file_authors = []

file_ids_title = []
file_titles = []

file_ids_fid = []
file_key_fid = []

bow_file_id = []
bow_file_word = []
bow_file_word_count = []
root_dir_nsf = f"{my_root}/Data/NSF_DATA/"
for subdir, dirs, files in os.walk(root_dir_nsf):
    for filename in files:
        f = os.path.join(subdir, filename)
        # Process authors 
        if f[-12:] == 'docauths.txt':
          with open(f, 'r', encoding='latin1') as file:
            docauths = file.readlines()
          for row in docauths:
            auth_data = row.split('\t')
            file_ids_auth.append(int(auth_data[0]))
            file_authors.append(auth_data[1].strip())

        # Process titles
        if f[-13:] == 'doctitles.txt':
          with open(f, 'r', encoding='latin1') as file:
            doctitles = file.readlines()
          for row in doctitles:
            title_data = row.split('\t')
            file_ids_title.append(int(title_data[0]))
            file_titles.append(title_data[1].strip())
        
        # Process idnsfid 
        if f[-11:] == 'idnsfid.txt':
          with open(f, 'r', encoding='latin1') as file:
            docfids = file.readlines()
          for row in docfids:
            fids_data = row.split('\t')
            file_ids_fid.append(int(fids_data[0]))
            file_key_fid.append(fids_data[1].strip())
        
        # Process BOW data 
        if f[-12:] == 'docwords.txt':
          with open(f, 'r', encoding='latin1') as file:
            docwords = file.readlines()
          for row in docwords:
            words_data = row.split(' ')
            bow_file_id.append(int(words_data[0]))
            bow_file_word.append(int(words_data[1].strip()))
            bow_file_word_count.append(int(words_data[2].strip()))


In [22]:
data = {'file_key':file_key, 'abstract': abstracts}
df_abstract = pd.DataFrame(data)

data_author = {'file_id':file_ids_auth, 'file_author': file_authors}
df_author = pd.DataFrame(data_author)

data_title = {'file_id':file_ids_title, 'file_title': file_titles}
df_title = pd.DataFrame(data_title)

data_fids = {'file_id':file_ids_fid, 'file_key': file_key_fid}
df_fid = pd.DataFrame(data_fids)

data_bow = {'file_id':bow_file_id, 'word_id': bow_file_word, 'word_freq': bow_file_word_count}
df_bow = pd.DataFrame(data_bow)

print(f"Length of abstract df: {len(df_abstract)}")
print(f"Length of author df: {len(df_author)}")
print(f"Length of title df: {len(df_title)}")
print(f"Length of fid df: {len(df_fid)}")
print(f"Length of BOW df: {len(df_bow)}")

Length of abstract df: 134616
Length of author df: 199475
Length of title df: 128818
Length of fid df: 128818
Length of BOW df: 10449902


In [23]:
df_abstract['abstract'][1]

'Excavations by Dr. Alan Simmons and his colleagues at the site ofAkrotiri-Aetokremnos, located on the Akrotiri Peninsula ofsouthern Cyprus, have greatly surprised both archaeologists andbiologists, because they seem to indicate the co-occurrence ofhumans and pygmy forms of both hippopotamus and elephant.  Datedto the ninth millennium BC, this stratified cave site containsboth stone tools as well as the broken bones of both species.These data also indicate that human settlement of the easternMediterranean islands occurred ca. 2,000 years earlier thanpreviously expected.  Evidence seems to indicate that a trueassociation exists between the bones and stone tools.  Thisimplies human hunting of these species and a probable human rolein their extinction.  In this project, Dr. Simmons and hiscolleagues will analyze the materials recovered during multipleseasons of field research.  Objectives include analysis of faunaland material culture assemblages, of site geomorphology,stratigraphy and pa

In [40]:
def push_data(df, df_name):
    df.to_parquet(f"{my_root}/Processed_Data/{df_name}.parquet.gzip", compression='gzip')

In [41]:
push_data(df_abstract, "df_abstract")
push_data(df_author, "df_author")
push_data(df_title, "df_title")
push_data(df_fid, "df_fid")
push_data(df_bow, "df_bow")